In [12]:
import boto3
import sagemaker
import os

In [13]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# location of stored sata
data_dir = 'modelling'
# S3 location
bucket = sagemaker_session.default_bucket()
# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/audio_emotion_detection'

## Train estimator

## Load and deploy saved model

An AWS blogpost about deployment of a keras model on SageMaker (https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/) gave me a good general idea how to proceed. First, I changed the training approach to save model in a format enabling deployment on SageMaker. The blogpost described a workaround to deploy a model, by using sagemaker.tensorflow.model.TensorFlowModel and simulating its creation with an empty train.py file. As I discovered from the Sagemaker documentation, there is now a better way to deploy directly from model artifacts (https://sagemaker.readthedocs.io/en/stable/using_tf.html#deploying-directly-from-model-artifacts).

In [14]:
!ls export/1

saved_model.pb	variables


## Deploy the trained model

framework_version was selected among the available at https://github.com/aws/sagemaker-python-sdk#tensorflow-sagemaker-estimators

TensorFlow Serving Model
https://sagemaker.readthedocs.io/en/stable/sagemaker.tensorflow.html

Specification of *framework_version='2.0.0'* alighns the TensorFlow version of the trained model with the version of the serving model. Otherwise, an error was thrown.


In [22]:
from sagemaker.tensorflow.serving import Model
model = Model(model_data = 's3://' + bucket + '/' + prefix + '/' + 'model.tar.gz', 
              role = role,
             framework_version='2.0.0')

In [23]:
%%time
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m4.xlarge')

--------------------------------------------------------------------------------------!CPU times: user 510 ms, sys: 32 ms, total: 542 ms
Wall time: 7min 13s


## Making predictions

Sending a preprocessed sound data (MFCC feature) to the endpoint results in probability prediction for each of the classes.

In [24]:
mfcc = [-5.5037286e+02,  4.8238766e+01,  1.8442627e+01,  6.5049920e+00,
        8.2730311e-01, -6.1545024e+00, -2.4476097e+01, -7.4760280e+00,
       -1.1998719e+01, -9.4280348e+00, -7.5956612e+00, -5.5952277e+00,
       -4.4645872e+00, -6.2611837e+00, -3.2972498e+00, -8.0563803e+00,
       -4.3905830e+00, -2.1417410e+00, -6.5392509e+00, -6.4882522e+00,
       -6.0786166e+00, -8.1123333e+00, -5.5041442e+00,  4.6315003e-02,
       -1.7572067e+00,  8.9478511e-01, -1.2644883e-01,  2.5937853e+00,
       -1.1436307e+00,  3.4961894e+00,  2.2928705e+00,  3.7813089e+00,
        4.3201895e+00,  1.7073919e+00,  1.4069341e+00, -1.3996479e-01,
       -3.3032024e-01,  1.0882645e+00, -5.8361721e-01,  1.6979592e+00]

In [44]:
input = {
  'instances': [mfcc]
}

In [45]:
result = predictor.predict(input)

In [47]:
result

{'predictions': [[0.0174345262,
   0.0509620309,
   0.61150676,
   0.00767038902,
   0.312426269]]}

In [ ]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))
